In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import torch
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def process_text(token):

  text = nltk.word_tokenize(token)

  text = [re.sub(r'%', ' بالمية', i) for i in text]
  text = [re.sub(r'[^\w\s]', ' ', i) for i in text]

  tashkil = re.compile(""" ّ    | # Tashdid
                       َ    | # Fatha
                       ً    | # Tanwin Fath
                       ُ    | # Damma
                       ٌ    | # Tanwin Damm
                       ِ    | # Kasra
                       ٍ    | # Tanwin Kasr
                       ْ    | # Sukun
                      ـ     # Tatwil/Kashida
                    """, re.VERBOSE)

  text = [re.sub(tashkil, '', i) for i in text]

  text = [re.sub(r'\bإلى\b', '', i) for i in text]

  text = [re.sub(r'\bمن\b', '', i) for i in text]

  text = [re.sub(r'\bعن\b', '', i) for i in text]

  text = [re.sub(r'\bعلى\b', '', i) for i in text]

  text = [re.sub(r'\bفي\b', '', i) for i in text]

  text = [re.sub(r'\bمع\b', '', i) for i in text]

  text = [re.sub(r'\bو\b', '', i) for i in text]

  text = [re.sub(r'\bجدا\b', '', i) for i in text]

  text = [re.sub(r'\bلن\b', '', i) for i in text]

  text = [re.sub(r'\bاي\b', '', i) for i in text]

  text = [re.sub(r'\bما\b', '', i) for i in text]

  text = [re.sub(r'\bلي\b', '', i) for i in text]

  text = [re.sub(r'\bلم\b', '', i) for i in text]

  text = [re.sub(r'^وال', '', i) for i in text]

  text = [re.sub(r'^هال', '', i) for i in text]

  text = [re.sub(r'^فال', '', i) for i in text]

  text = [re.sub(r'^بال', '', i) for i in text]

  text = [re.sub(r'^عال', '', i) for i in text]

  text = [re.sub(r'^كال', '', i) for i in text]

  text = [re.sub(r'^ال', '', i) for i in text]

  text = [re.sub(r'^و', '', i) for i in text]

  text = [re.sub("[إأٱآا]", "ا", i) for i in text]

  text = [re.sub("ى", "ي", i) for i in text]

  text = [re.sub("ة", "ه", i) for i in text]

  text = [re.sub("ؤ", "و", i) for i in text]

  text = [re.sub("ئ", "ي", i) for i in text]

  processed_text = ' '.join(text)

  text = re.sub(r'\s+', ' ', processed_text)

  text = re.sub(r'(.)\1+', r"\1\1", text)

  return text

In [ ]:
df_test = pd.read_excel("/content/News_test.xlsx")
df_test.head()

,Type,News
0,politics,مئات القتلى في فيضانات أوروبا
1,politics,جولة مفاوضات جديدة تفشل في كسر جمود الأزمة الق...
2,politics,"القضاء الفرنسي يفتح تحقيقا حول ""بيغاسوس"" بشأن ..."
3,politics,وزير الدفاع الإسرائيلي يزور فرنسا لبحث قضية ″ب...
4,politics,بدأ في عدة دول سريان الحظر الشامل للحد من انتش...


In [ ]:
processing_text_test = [process_text(text) for text in df_test['News']]
processing_text_test

['ميات قتلي فيضانات اوروبا',
 'جوله مفاوضات جديده تفشل كسر جمود ازمه قبرصيه',
 'قضاء فرنسي يفتح تحقيقا حول بيغاسوس بشان تجسس صحافيين',
 'زير دفاع اسراييلي يزور فرنسا لبحث قضيه بيغاسوس ',
 'بدا عده دول سريان حظر شامل للحد انتشار فيروس كورونا',
 'حكومه اردنيه تصدر اجراءات مشدده للحد تفشي فيروس كورونا',
 'حركه طالبان تطلق سراح عشرات قوات حكومه افغانيه',
 'عشرات قتلي طالبان قوات حكوميه افغانستان',
 'سودان يجدد رفضه اجرءات اثيوبيا احاديه بعد اعلان اخيره استكمال تعبيه سد نهضه',
 'رييس زراء اثيوبي يوجه رساله جديده مصر سودان بعد تمام ملء ثاني لسد نهضه',
 'مستشاره المانيه تزور منطقه تي شهدت كارثه فيضانات بلاد',
 'ميركل تصف خساير فيضانات مخيفه',
 'سلطات امريكيه تعتقل مستشار للرييس اميركي سابق بتهمه عماله للامارات',
 'زاره عدل امريكيه ترفض تعليق تقارير تحقيقات ت جري سفير دوله امارات',
 'صرحت روسيا تري ان طالبان لا تمثل تهديدا لاسيا سطي',
 'روسيا تقدم مساعدات لطالبان',
 'شركه تركيه للصناعه طايرات تكشف اصدار جديد طايراتها',
 'شركه بايكار تعتزم بناء مركز لتجميع طايرات مسيره',
 'اعلن رييس اميركي تمدي

In [ ]:
df_test['processed_text'] = processing_text_test
df_test.drop(columns = ['News'], inplace = True)
df_test.head()

,Type,processed_text
0,politics,ميات قتلي فيضانات اوروبا
1,politics,جوله مفاوضات جديده تفشل كسر جمود ازمه قبرصيه
2,politics,قضاء فرنسي يفتح تحقيقا حول بيغاسوس بشان تجسس ص...
3,politics,زير دفاع اسراييلي يزور فرنسا لبحث قضيه بيغاسوس
4,politics,بدا عده دول سريان حظر شامل للحد انتشار فيروس ك...


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_test['processed_text'])
X_seq = tokenizer.texts_to_sequences(df_test['processed_text'])
max_sequence_length = max([len(seq) for seq in X_seq])
max_sequence_length

22

In [ ]:
X_full_test = df_test['processed_text']
y_full_test = df_test['Type']

In [ ]:
lr = LabelEncoder()
y_ecod_test = lr.fit_transform(df_test['Type'])

In [ ]:
X_train_test, X_test, y_train_test, y_test = train_test_split(X_full_test, y_full_test, test_size = 0.2, random_state = 42)

X_train_test.reset_index(drop = True, inplace = True)
X_test.reset_index(drop = True, inplace = True)

# **BERT Embedding with Logistic Regression**

In [ ]:
model_name = "aubmindlab/bert-base-arabert"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/717k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
bert_embeddings_train = []
for j in X_train_test:
  inputs = tokenizer(j, padding = True, truncation = True, return_tensors = 'pt', max_length = max_sequence_length)
  with torch.no_grad():
    outputs = bert_model(**inputs)
  embeddings = outputs.last_hidden_state[:, 0, :]
  bert_embeddings_train.append(embeddings.numpy())

In [ ]:
bert_embeddings_train_np = np.array(bert_embeddings_train)
bert_embeddings_train_np.shape

(800, 1, 768)

In [ ]:
bert_embeddings_test = []
for i in X_test:
  inputs = tokenizer(i, padding = True, truncation = True, return_tensors = 'pt', max_length = max_sequence_length)
  with torch.no_grad():
    outputs = bert_model(**inputs)
  embeddings = outputs.last_hidden_state[:, 0, :]
  bert_embeddings_test.append(embeddings.numpy())

In [ ]:
bert_embeddings_test_np = np.array(bert_embeddings_test)
bert_embeddings_test_np.shape

(200, 1, 768)

In [ ]:
bert_embeddings_train_lr = bert_embeddings_train_np[:,0,:]
bert_embeddings_test_lr = bert_embeddings_test_np[:,0,:]

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(bert_embeddings_train_lr, y_train_test)

y_pred_lr = lr.predict(bert_embeddings_test_lr)

print("Classification Report:")
print(classification_report(y_test, y_pred_lr, target_names=lr.classes_))

Classification Report:
              precision    recall  f1-score   support

    economic       0.89      0.83      0.86        47
    politics       0.88      0.95      0.91        96
       sport       1.00      1.00      1.00        38
        tech       0.93      0.74      0.82        19

    accuracy                           0.91       200
   macro avg       0.93      0.88      0.90       200
weighted avg       0.91      0.91      0.91       200



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# **BERT Model**

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_full_test)

In [ ]:
X_train_test, X_test, y_train_test, y_test = train_test_split(X_full_test, y, test_size = 0.2, random_state = 42)

In [ ]:
# Preprocess the data
tokenizer = BertTokenizer.from_pretrained(model_name)
encoded_data_train = tokenizer.batch_encode_plus(X_train_test.tolist(), add_special_tokens=True, return_attention_mask=True, padding=True,
                                           truncation=True, max_length=27, return_tensors='pt')


encoded_data_test = tokenizer.batch_encode_plus(X_test.tolist(), add_special_tokens=True, return_attention_mask=True, padding=True,
                                           truncation=True, max_length=27, return_tensors='pt')


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train_test)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(y_test)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_), output_attentions=False, output_hidden_states=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
input_ids_train = input_ids_train.to(device)
attention_masks_train = attention_masks_train.to(device)
labels_train = labels_train.to(device)
input_ids_test = input_ids_test.to(device)
attention_masks_test = attention_masks_test.to(device)
labels_test = labels_test.to(device)

# Define the training parameters
batch_size = 32
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Train the model
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(0, input_ids_train.size(0), batch_size):
        batch_input_ids = input_ids_train[i:i+batch_size]
        batch_attention_masks = attention_masks_train[i:i+batch_size]
        batch_labels = labels_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / (input_ids_train.size(0) // batch_size)
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss:.4f}')


# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(input_ids_test, attention_mask=attention_masks_test)
    predictions = torch.argmax(outputs.logits, dim=1).flatten()
    accuracy = (predictions == labels_test).cpu().numpy().mean()

print("Accuracy:", accuracy)

labels_test_cpu = labels_test.cpu().numpy()  # Move labels to CPU if necessary
predictions_cpu = predictions.cpu().numpy()  # Move predictions to CPU if necessary
report = classification_report(labels_test_cpu, predictions_cpu, target_names=label_encoder.classes_)
print(report)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5, Training Loss: 1.1968
Epoch 2/5, Training Loss: 0.8982
Epoch 3/5, Training Loss: 0.5900
Epoch 4/5, Training Loss: 0.3798
Epoch 5/5, Training Loss: 0.2537
Accuracy: 0.91
              precision    recall  f1-score   support

    economic       0.93      0.83      0.88        47
    politics       0.89      0.97      0.93        96
       sport       0.95      1.00      0.97        38
        tech       0.92      0.63      0.75        19

    accuracy                           0.91       200
   macro avg       0.92      0.86      0.88       200
weighted avg       0.91      0.91      0.91       200

